In [1]:
# 05_neural_baseline.ipynb — Task 3.1 (TensorFlow required; Kaggle + Local)

# - Loads data (auto-detect paths)
# - Keras: TextVectorization + Embedding + CNN + BiGRU
# - Early stopping + threshold tuning (macro-F1)
# - Saves Kaggle-ready /kaggle/working/submission.csv and local submissions/submission.csv

# ========= 0) Imports & environment info =========
import sys, os, glob, re, json, warnings
import numpy as np
import pandas as pd

print("Python:", sys.version)
print("NumPy :", np.__version__)
print("Pandas:", pd.__version__)

# Output locations
IS_KAGGLE = os.path.exists("/kaggle/input")
KAGGLE_WORKING = "/kaggle/working" if IS_KAGGLE else None
OUT_KAGGLE = os.path.join(KAGGLE_WORKING, "submission.csv") if IS_KAGGLE else None
os.makedirs("submissions", exist_ok=True)
OUT_LOCAL = "submissions/submission.csv"
os.makedirs("results", exist_ok=True)

# ========= 1) Require TensorFlow (fail fast if unavailable) =========
try:
    import tensorflow as tf
    print("TensorFlow:", tf.__version__)
    print("Devices:", tf.config.list_physical_devices())
except Exception as e:
    raise RuntimeError(
        "TensorFlow is not available in this environment.\n"
        "Kaggle: Settings → Environment → set Image=TensorFlow and Accelerator=GPU, Internet=OFF, then Save Version → Run All.\n"
        "Local (Apple Silicon): install tensorflow-macos; Local (Intel): install tensorflow==2.15.x."
    ) from e

# Optional: set sensible CPU thread limits to avoid oversubscription on some hosts
os.environ.setdefault("OMP_NUM_THREADS", "4")
tf.config.set_soft_device_placement(True)

# ========= 2) Locate data (Kaggle-first, then local fallbacks) =========
KAGGLE_DIR = "/kaggle/input/jigsaw-agile-community-rules"
CANDIDATE_DIRS = [
    ".", "..", "../..", "../../..",
    "data/raw", "../data/raw", "../../data/raw",
    "jigsaw-agile-community-rules", "../jigsaw-agile-community-rules"
]

def _candidate_paths(filename: str):
    paths = []
    if os.path.exists(KAGGLE_DIR):
        paths.append(os.path.join(KAGGLE_DIR, filename))
    for d in CANDIDATE_DIRS:
        paths.append(os.path.join(d, filename))
    paths.extend(glob.glob(f"**/{filename}", recursive=True))
    # Deduplicate existing
    seen, out = set(), []
    for p in paths:
        ap = os.path.abspath(p)
        if ap not in seen and os.path.exists(ap):
            seen.add(ap); out.append(ap)
    return out

def read_first_csv(filename: str):
    found = _candidate_paths(filename)
    if not found:
        raise FileNotFoundError(f"Could not find {filename} in Kaggle folder or local fallbacks.")
    print(f"📄 Loading {filename} from: {found[0]}")
    return pd.read_csv(found[0])

train_df = read_first_csv("train.csv")
test_df  = read_first_csv("test.csv")
sample   = read_first_csv("sample_submission.csv")

print("Train shape:", train_df.shape)
print("Test  shape:", test_df.shape)
print("Sample shape:", sample.shape)

# ========= 3) Robust column detection =========
TEXT_COL = next((c for c in ["comment_text", "body", "text"] if c in train_df.columns), None)
TARGET_COL = next((c for c in ["rule_violation", "target", "label"] if c in train_df.columns), None)
ID_COL = sample.columns[0]
TARGET_OUT = sample.columns[1]

assert TEXT_COL is not None, "No text column found (expected comment_text/body/text)."
assert TARGET_COL is not None, "No target column found (expected rule_violation/target/label)."
print(f"TEXT_COL  = {TEXT_COL}")
print(f"TARGET_COL= {TARGET_COL}")
print(f"ID_COL    = {ID_COL} | TARGET_OUT = {TARGET_OUT}")

# ========= 4) Prepare data & utilities =========
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, confusion_matrix, classification_report

SEED = 42
rng = np.random.default_rng(SEED)

X_text = train_df[TEXT_COL].fillna("").astype(str).values
y = train_df[TARGET_COL].astype(int).values
X_test_text = test_df[TEXT_COL].fillna("").astype(str).values

# stratified split for validation
X_tr_text, X_va_text, y_tr, y_va = train_test_split(
    X_text, y, test_size=0.2, random_state=SEED, stratify=y
)

def compute_class_weights(y_array):
    # balanced weights: N / (2 * count_class)
    n = len(y_array)
    pos = int((y_array == 1).sum())
    neg = n - pos
    return {0: n/(2*neg), 1: n/(2*pos)}

CLASS_WEIGHTS = compute_class_weights(y_tr)
print("Class weights:", CLASS_WEIGHTS)

# ========= 5) Build tf.data + TextVectorization pipeline (TEXT ONLY for now) =========
print("Step 5: Building TextVectorization…")
AUTOTUNE = tf.data.AUTOTUNE
MAX_TOKENS = 30000
SEQ_LEN = 200
HAS_GPU = bool(tf.config.list_physical_devices('GPU'))
BATCH = 64 if HAS_GPU else 32
EPOCHS = 16  # early stopping will halt earlier
print(f"  HAS_GPU={HAS_GPU} | BATCH={BATCH}")

# Raw tf.data datasets (text only)
ds_tr_raw = tf.data.Dataset.from_tensor_slices((X_tr_text, y_tr))
ds_va_raw = tf.data.Dataset.from_tensor_slices((X_va_text, y_va))
ds_te_raw = tf.data.Dataset.from_tensor_slices(X_test_text)

# Vectoriser (adapt on train only)
text_vec = tf.keras.layers.TextVectorization(
    max_tokens=MAX_TOKENS,
    output_mode="int",
    output_sequence_length=SEQ_LEN,
    standardize="lower_and_strip_punctuation",
    split="whitespace"
)
text_vec.adapt(ds_tr_raw.map(lambda x, y: x).batch(256))
print("  TextVectorization adapted.")

# ========= 6) Load engineered features (or compute), then split with same indices =========
print("Step 6: Loading / computing engineered features…")
TR_FEATS_PKL = "data/processed/train_features.pkl"
TE_FEATS_PKL = "data/processed/test_features.pkl"

def extract_features_v2(df, text_col):
    # (Paste your 03_feature_engineering extractor here if not already imported)
    import re
    s = df[text_col].fillna("").astype(str)
    feats = pd.DataFrame(index=df.index)
    _safe_div = lambda a,b: a/np.maximum(b,1)
    NEG_WORDS = {"ban","banned","remove","removed","delete","deleted","violation","warn","warning","report","flag","hate","toxic","idiot","stupid","dumb","trash","nonsense","shut","shutup","racist","sexist","harass","abuse","spam","brigade","rule","rules","automod","mod","moderator"}
    POS_WORDS = {"please","thanks","thank","appreciate","sorry","kindly","cheers"}
    QUESTION_WORDS = {"why","how","what","when","where","which","who"}
    NEGATIONS = {"not","no","never","n't"}
    EMOJI_RE = re.compile(r"[\U0001F300-\U0001FAFF]")
    REPEAT_CHAR_RE = re.compile(r"(.)\1{2,}")
    MD_LINK_RE = re.compile(r"\[[^\]]+\]\([^)]+\)")
    def _count_tokens(text, vocab): 
        toks = text.lower().split(); 
        return sum(t in vocab for t in toks)
    feats["char_count"]  = s.str.len()
    feats["word_count"]  = s.str.split().str.len().astype("int64")
    feats["uniq_word_count"] = s.apply(lambda x: len(set(x.lower().split())))
    feats["lexical_diversity"] = _safe_div(feats["uniq_word_count"], feats["word_count"])
    feats["avg_word_len"] = _safe_div(feats["char_count"], feats["word_count"])
    feats["upper_count"] = s.str.count(r"[A-Z]")
    feats["caps_ratio"]  = _safe_div(feats["upper_count"], feats["char_count"])
    feats["all_caps_words"] = s.str.count(r"\b[A-Z]{2,}\b")
    feats["excl_count"]  = s.str.count("!")
    feats["ques_count"]  = s.str.count(r"\?")
    feats["dots_count"]  = s.str.count(r"\.")
    feats["ellipsis_count"] = s.str.count(r"\.\.\.")
    feats["multi_excl"]  = s.str.count(r"!!+")
    feats["multi_ques"]  = s.str.count(r"\?\?+")
    feats["mix_punct"]   = s.str.count(r"[!?]{2,}")
    feats["punct_ratio"] = _safe_div(feats["excl_count"]+feats["ques_count"]+feats["dots_count"], feats["char_count"])
    feats["repeat_char"] = s.apply(lambda x: len(REPEAT_CHAR_RE.findall(x)))
    feats["has_user_mention"]      = s.str.contains(r"u/\w+", case=False, regex=True).astype("int8")
    feats["has_subreddit_mention"] = s.str.contains(r"r/\w+", case=False, regex=True).astype("int8")
    feats["quote_count"]           = s.str.count(r"^>|\n>", flags=re.MULTILINE)
    feats["code_ticks"]            = s.str.count(r"`")
    feats["md_links"]              = s.apply(lambda x: len(MD_LINK_RE.findall(x))).astype("int16")
    feats["has_url"]     = s.str.contains(r"http[s]?://", case=False, regex=True).astype("int8")
    feats["email_count"] = s.str.count(r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}")
    feats["digit_count"] = s.str.count(r"\d")
    feats["num_ratio"]   = _safe_div(feats["digit_count"], feats["char_count"])
    feats["emoji_count"] = s.apply(lambda x: len(EMOJI_RE.findall(x))).astype("int16")
    feats["neg_lex_count"] = s.apply(lambda x: _count_tokens(x, NEG_WORDS)).astype("int16")
    feats["pos_lex_count"] = s.apply(lambda x: _count_tokens(x, POS_WORDS)).astype("int16")
    feats["neg_lex_ratio"] = _safe_div(feats["neg_lex_count"], feats["word_count"])
    feats["pos_lex_ratio"] = _safe_div(feats["pos_lex_count"], feats["word_count"])
    feats["you_count"] = s.str.count(r"\byou\b", flags=re.IGNORECASE)
    feats["i_count"]   = s.str.count(r"\bi\b",   flags=re.IGNORECASE)
    feats["you_ratio"] = _safe_div(feats["you_count"], feats["word_count"])
    feats["i_ratio"]   = _safe_div(feats["i_count"],   feats["word_count"])
    feats["wh_q_count"]  = s.apply(lambda x: _count_tokens(x, QUESTION_WORDS)).astype("int16")
    feats["negate_count"]= s.apply(lambda x: _count_tokens(x, NEGATIONS)).astype("int16")
    feats["starts_with_quote"] = s.str.match(r'^\s*["\']').astype("int8")
    feats["ends_with_q"]       = s.str.endswith("?").astype("int8")
    feats["ends_with_excl"]    = s.str.endswith("!").astype("int8")
    feats["excl_ratio"] = _safe_div(feats["excl_count"], feats["char_count"])
    feats["ques_ratio"] = _safe_div(feats["ques_count"], feats["char_count"])
    feats = feats.replace([np.inf, -np.inf], 0).fillna(0)
    return feats

if os.path.exists(TR_FEATS_PKL) and os.path.exists(TE_FEATS_PKL):
    train_features = pd.read_pickle(TR_FEATS_PKL)
    test_features  = pd.read_pickle(TE_FEATS_PKL)
    print("  Loaded cached features.")
else:
    train_features = extract_features_v2(train_df, TEXT_COL)
    test_features  = extract_features_v2(test_df,  TEXT_COL)
    os.makedirs("data/processed", exist_ok=True)
    train_features.to_pickle(TR_FEATS_PKL)
    test_features.to_pickle(TE_FEATS_PKL)
    print("  Computed and cached features.")

from sklearn.preprocessing import StandardScaler
feat_scaler = StandardScaler()
feats_tr_all = feat_scaler.fit_transform(train_features.values.astype(np.float32))
feats_te     = feat_scaler.transform(test_features.values.astype(np.float32))

# Use *the same* split indices as for text
X_idx = np.arange(len(X_text))
idx_tr, idx_va, _, _ = train_test_split(X_idx, y, test_size=0.2, random_state=SEED, stratify=y)
feats_tr_split = feats_tr_all[idx_tr]
feats_va_split = feats_tr_all[idx_va]
print("  Feats shapes:", feats_tr_split.shape, feats_va_split.shape, feats_te.shape)

# ========= 7) Build tf.data that yields (text, aux_feats) =========
print("Step 7: Building tf.data with auxiliary features…")
def make_ds_with_feats(x_arr, feats_arr, y_arr=None, train=False):
    if y_arr is None:
        ds = tf.data.Dataset.from_tensor_slices((x_arr, feats_arr))
        if train:
            ds = ds.shuffle(len(x_arr), seed=SEED)
        ds = ds.batch(BATCH).map(
            lambda txt, f: (text_vec(txt), f),
            num_parallel_calls=AUTOTUNE
        )
    else:
        ds = tf.data.Dataset.from_tensor_slices((x_arr, feats_arr, y_arr))
        if train:
            ds = ds.shuffle(len(x_arr), seed=SEED)
        ds = ds.batch(BATCH).map(
            lambda txt, f, y: ((text_vec(txt), f), y),
            num_parallel_calls=AUTOTUNE
        )
    return ds.prefetch(AUTOTUNE)

ds_tr = make_ds_with_feats(X_tr_text, feats_tr_split, y_tr, train=True)
ds_va = make_ds_with_feats(X_va_text, feats_va_split, y_va, train=False)
ds_te = make_ds_with_feats(X_test_text, feats_te, y_arr=None, train=False)
print("  tf.data ready.")

# --- Sanity check: make sure the dataset yields one batch ---
for batch in ds_tr.take(1):
    (tok_ids, aux_feats), yb = batch
    print("Sample batch:", tok_ids.shape, aux_feats.shape, yb.shape)
# Expect something like: (32 or 64, SEQ_LEN), (32 or 64, 42), (32 or 64,)


# ========= 8) Build model (dual-branch text + aux feats) =========
print("Step 8: Building model…")
tf.random.set_seed(SEED)
vocab_size   = MAX_TOKENS + 2
embed_dim    = 128
gru_units    = 96
dropout_rate = 0.35
l2_reg       = tf.keras.regularizers.l2(1e-5)

tok_in = tf.keras.Input(shape=(SEQ_LEN,), dtype="int32", name="tok_ids")
emb = tf.keras.layers.Embedding(vocab_size, embed_dim, mask_zero=False,
                                embeddings_regularizer=l2_reg, name="emb")(tok_in)

# RNN branch
rnn = tf.keras.layers.Bidirectional(
    tf.keras.layers.GRU(gru_units, return_sequences=True, dropout=0.2, recurrent_dropout=0.15),
    name="bigru_seq"
)(emb)
rnn = tf.keras.layers.GlobalMaxPooling1D(name="rnn_gmp")(rnn)

# CNN branch
cnn = tf.keras.layers.SpatialDropout1D(0.2, name="spatial_dropout")(emb)
cnn = tf.keras.layers.Conv1D(128, 3, padding="same", activation="relu", kernel_regularizer=l2_reg)(cnn)
cnn = tf.keras.layers.Conv1D(128, 5, padding="same", activation="relu", kernel_regularizer=l2_reg)(cnn)
gmp = tf.keras.layers.GlobalMaxPooling1D(name="cnn_gmp")(cnn)
gap = tf.keras.layers.GlobalAveragePooling1D(name="cnn_gap")(cnn)
cnn = tf.keras.layers.Concatenate(name="cnn_pool")([gmp, gap])

# Aux (engineered) features
aux_in = tf.keras.Input(shape=(feats_tr_all.shape[1],), dtype="float32", name="aux_feats")
aux_h  = tf.keras.layers.Dense(64, activation="relu", kernel_regularizer=l2_reg)(aux_in)
aux_h  = tf.keras.layers.Dropout(0.2)(aux_h)

# Fuse and head
h = tf.keras.layers.Concatenate(name="fusion")([rnn, cnn, aux_h])
h = tf.keras.layers.Dropout(dropout_rate)(h)
h = tf.keras.layers.Dense(192, activation="relu", kernel_regularizer=l2_reg)(h)
h = tf.keras.layers.Dropout(dropout_rate)(h)
out = tf.keras.layers.Dense(1, activation="sigmoid", name="out")(h)

model = tf.keras.Model(inputs=[tok_in, aux_in], outputs=out, name="hybrid_text_feats")
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=2e-3),
    loss="binary_crossentropy",
    metrics=[tf.keras.metrics.AUC(name="auc"),
             tf.keras.metrics.Precision(name="prec"),
             tf.keras.metrics.Recall(name="rec")]
)

# --- Warm-up: force a single forward pass to compile kernels/graph ---
print("Warm-up: building graph with a single batch …")
_ = model.predict(ds_tr.take(1), verbose=0)
print("Warm-up predict done. Starting training…")

model.summary()

# ========= 9) Train with callbacks =========
print("Step 9: Training…")
ckpt_dir = (KAGGLE_WORKING if IS_KAGGLE else "models")
os.makedirs(ckpt_dir, exist_ok=True)
ckpt_path = os.path.join(ckpt_dir, "tf_neural_baseline_v2.keras")

callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=3, restore_best_weights=True, verbose=1),
    tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=2, min_lr=5e-5, verbose=1),
    tf.keras.callbacks.ModelCheckpoint(ckpt_path, monitor="val_loss", save_best_only=True, verbose=1)
]
history = model.fit(
    ds_tr,
    validation_data=ds_va,
    epochs=EPOCHS,
    class_weight=CLASS_WEIGHTS,
    callbacks=callbacks,
    verbose=1   # was 2
)

print("  Training done.")

# ========= 10) Threshold tuning & evaluation =========
print("Step 10: Threshold tuning…")
va_probs = model.predict(ds_va, verbose=0).ravel()
thr_grid = np.linspace(0.30, 0.70, 81)
f1s = [f1_score(y_va, (va_probs >= t).astype(int), average="macro") for t in thr_grid]
best_idx = int(np.argmax(f1s))
best_threshold = float(thr_grid[best_idx])
val_f1_macro = float(f1s[best_idx])
print(f"  Best threshold = {best_threshold:.3f} | Val F1(macro) = {val_f1_macro:.4f}")

y_pred_va = (va_probs >= best_threshold).astype(int)
print("Validation confusion matrix:\n", confusion_matrix(y_va, y_pred_va))
print(classification_report(y_va, y_pred_va, digits=4))

# ========= 11) Predict test & build submission =========
print("Step 11: Predicting test & writing submission…")

# 11a) Explicit tokenisation to arrays (avoids Dataset tuple quirks)
X_te_tok = text_vec(tf.constant(X_test_text))
# If you're low on memory, you can .numpy() this after prediction; Keras accepts EagerTensors.
# X_te_tok_np = X_te_tok.numpy()

# 11b) Predict with explicit two-input list [tokens, features]
test_probs = model.predict([X_te_tok, feats_te], batch_size=BATCH, verbose=0).ravel()
test_pred  = (test_probs >= best_threshold).astype(int)

submission = sample.copy()
submission[TARGET_OUT] = test_pred.astype(int)

# Validation of format (unchanged)
errors = []
if list(submission.columns) != list(sample.columns):
    errors.append(f"Columns mismatch. Expected {list(sample.columns)}, got {list(submission.columns)}")
if len(submission) != len(sample):
    errors.append(f"Row count mismatch. Expected {len(sample)}, got {len(submission)}")
if not submission[ID_COL].equals(sample[ID_COL]):
    if set(submission[ID_COL]) != set(sample[ID_COL]):
        missing = list(sorted(set(sample[ID_COL]) - set(submission[ID_COL])))[:5]
        extra   = list(sorted(set(submission[ID_COL]) - set(sample[ID_COL])))[:5]
        errors.append(f"ID set differs. Missing: {missing} | Extra: {extra}")
    else:
        errors.append("ID order differs from sample. Must match sample_submission order.")
if submission[TARGET_OUT].isna().any():
    errors.append("Target has NaNs.")
u = set(np.unique(submission[TARGET_OUT]))
if not u.issubset({0,1}):
    errors.append(f"Target invalid values {sorted(u)}; must be 0/1.")
if errors:
    print("❌ Submission invalid:")
    for e in errors: print(" -", e)
    raise SystemExit(1)

if IS_KAGGLE:
    submission.to_csv(OUT_KAGGLE, index=False)
    print(f"✅ Saved Kaggle file: {OUT_KAGGLE}")
submission.to_csv(OUT_LOCAL, index=False)
print(f"✅ Saved local copy : {OUT_LOCAL}")


# ========= 12) Log run info =========
print("Step 12: Logging run info…")
run_info = {
    "task": "3.1_neural_baseline_hybrid",
    "model": "Hybrid CNN+BiGRU + 42 feats",
    "val_f1_macro": float(val_f1_macro),
    "best_threshold": float(best_threshold),
    "seed": SEED,
    "params": {
        "max_tokens": MAX_TOKENS, "seq_len": SEQ_LEN, "batch": BATCH, "epochs": EPOCHS,
        "embed_dim": embed_dim, "gru_units": gru_units,
        "dropout_rate": dropout_rate, "l2": 1e-5
    }
}
with open("results/run_05_neural_baseline.json","w") as f:
    json.dump(run_info, f, indent=2)

print("\n✅ Done.")


Python: 3.12.7 | packaged by Anaconda, Inc. | (main, Oct  4 2024, 08:22:19) [Clang 14.0.6 ]
NumPy : 1.26.4
Pandas: 2.2.3
TensorFlow: 2.16.2
Devices: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
📄 Loading train.csv from: /Users/michaelmaclennan/Documents/Learning & Education/2025-04 AI & ML/jigsaw-competition/data/raw/train.csv
📄 Loading test.csv from: /Users/michaelmaclennan/Documents/Learning & Education/2025-04 AI & ML/jigsaw-competition/data/raw/test.csv
📄 Loading sample_submission.csv from: /Users/michaelmaclennan/Documents/Learning & Education/2025-04 AI & ML/jigsaw-competition/data/raw/sample_submission.csv
Train shape: (2029, 9)
Test  shape: (10, 8)
Sample shape: (10, 2)
TEXT_COL  = body
TARGET_COL= rule_violation
ID_COL    = row_id | TARGET_OUT = rule_violation
Class weights: {0: 1.0169172932330828, 1: 0.9836363636363636}
Step 5: Building TextVectorization…
  HAS_GPU=True | BATCH=64


2025-09-19 13:26:22.710511: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Max
2025-09-19 13:26:22.710538: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 32.00 GB
2025-09-19 13:26:22.710542: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 12.48 GB
2025-09-19 13:26:22.710586: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-09-19 13:26:22.710612: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2025-09-19 13:26:22.876119: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


  TextVectorization adapted.
Step 6: Loading / computing engineered features…
  Loaded cached features.
  Feats shapes: (1623, 42) (406, 42) (10, 42)
Step 7: Building tf.data with auxiliary features…
  tf.data ready.
Sample batch: (64, 200) (64, 42) (64,)
Step 8: Building model…


2025-09-19 13:26:23.079124: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Warm-up: building graph with a single batch …


...

Step 9: Training…
Epoch 1/16
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 125s/step - auc: 0.5607 - loss: 0.7281 - prec: 0.5601 - rec: 0.5746  

2025-09-19 14:20:38.567396: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] PluggableGraphOptimizer failed: INVALID_ARGUMENT: Failed to deserialize the `graph_buf`.



Epoch 1: val_loss improved from None to 0.62050, saving model to models/tf_neural_baseline_v2.keras
26/26 ━━━━━━━━━━━━━━━━━━━━ 3280s 126s/step - auc: 0.6207 - loss: 0.7011 - prec: 0.5965 - rec: 0.6145 - val_auc: 0.7356 - val_loss: 0.6205 - val_prec: 0.6446 - val_rec: 0.7573 - learning_rate: 0.0020
Epoch 2/16
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 126s/step - auc: 0.7769 - loss: 0.5839 - prec: 0.6964 - rec: 0.8023  
Epoch 2: val_loss improved from 0.62050 to 0.56992, saving model to models/tf_neural_baseline_v2.keras
26/26 ━━━━━━━━━━━━━━━━━━━━ 3317s 128s/step - auc: 0.8012 - loss: 0.5559 - prec: 0.7081 - rec: 0.7939 - val_auc: 0.8391 - val_loss: 0.5699 - val_prec: 0.8333 - val_rec: 0.5097 - learning_rate: 0.0020
Epoch 3/16
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 123s/step - auc: 0.9461 - loss: 0.3260 - prec: 0.8999 - rec: 0.8257  
Epoch 3: val_loss did not improve from 0.56992
26/26 ━━━━━━━━━━━━━━━━━━━━ 3225s 124s/step - auc: 0.9505 - loss: 0.2977 - prec: 0.8953 - rec: 0.8703 - val_auc: 0.8415 - val_loss

2025-09-19 17:58:41.767512: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] PluggableGraphOptimizer failed: INVALID_ARGUMENT: Failed to deserialize the `graph_buf`.


  Best threshold = 0.320 | Val F1(macro) = 0.7755
Validation confusion matrix:
 [[149  51]
 [ 40 166]]
              precision    recall  f1-score   support

           0     0.7884    0.7450    0.7661       200
           1     0.7650    0.8058    0.7849       206

    accuracy                         0.7759       406
   macro avg     0.7767    0.7754    0.7755       406
weighted avg     0.7765    0.7759    0.7756       406

Step 11: Predicting test & writing submission…
✅ Saved local copy : submissions/submission.csv
Step 12: Logging run info…

✅ Done.
